## **MKT 7317 Problem Set 1**

#### Jin Miao jxm190071@utdallas.edu

Before getting into start working on this problem set, among other things, please get yourselves familiarized with (i) how to use **@jit** of the **numba** library, and (ii) how to parallelize your code in Python using **joblib**:

In [1]:
from joblib import Parallel, delayed
import time
def f(x):
    time.sleep(2)
    return x**2
results = Parallel(n_jobs=8)(delayed(f)(i) for i in range(10))
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## _1 (Python) Calculating the OLS Estimator for a Multiple Regression_

(a) Load **“cps09mar.csv”** into memory (use either polars or pandas)

In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv("/home/bizmia/Desktop/Joonhwi/Task1/cps09mar.csv")
data

,age,female,hisp,education,earnings,hours,week,union,uncov,region,race,marital
0,52,0,0,12,146000,45,52,0,0,1,1,1
1,38,0,0,18,50000,45,52,0,0,1,1,1
2,38,0,0,14,32000,40,51,0,0,1,1,1
3,41,1,0,13,47000,40,52,0,0,1,1,1
4,42,0,0,13,161525,50,52,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
50737,58,1,0,11,30000,40,52,0,0,4,4,6
50738,62,1,0,16,35000,40,52,0,0,4,4,6
50739,58,0,0,12,75000,50,52,0,0,4,1,1
50740,45,1,0,12,40000,60,52,0,0,4,1,1


(b) Generate a column of ones, and generate a variable **log_wage** by taking natural loga-
rithm of the variable earnings.

In [3]:
data['ones'] = 1
data['log_wage'] = np.log(data.earnings)
data

,age,female,hisp,education,earnings,hours,week,union,uncov,region,race,marital,ones,log_wage
0,52,0,0,12,146000,45,52,0,0,1,1,1,1,11.891362
1,38,0,0,18,50000,45,52,0,0,1,1,1,1,10.819778
2,38,0,0,14,32000,40,51,0,0,1,1,1,1,10.373491
3,41,1,0,13,47000,40,52,0,0,1,1,1,1,10.757903
4,42,0,0,13,161525,50,52,1,0,1,1,1,1,11.992415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50737,58,1,0,11,30000,40,52,0,0,4,4,6,1,10.308953
50738,62,1,0,16,35000,40,52,0,0,4,4,6,1,10.463103
50739,58,0,0,12,75000,50,52,0,0,4,1,1,1,11.225243
50740,45,1,0,12,40000,60,52,0,0,4,1,1,1,10.596635


(c) Extract the data matrices **y**, **X** as numpy array.

In [4]:
y = data.log_wage.to_numpy()
print(y)
print(type(y))
X = data[['ones','female','education', 'hours']].to_numpy()
print(X)
print(type(X))

[11.8913619  10.81977828 10.37349118 ... 11.22524339 10.59663473
 11.00209984]
<class 'numpy.ndarray'>
[[ 1  0 12 45]
 [ 1  0 18 45]
 [ 1  0 14 40]
 ...
 [ 1  0 12 50]
 [ 1  1 12 60]
 [ 1  0 11 40]]
<class 'numpy.ndarray'>


Now Consider the population regression function:
$$y_i = x_i^{i} \beta + \mu_i$$
where $y_i$ corresponds to the log_wage variable and $x_i$ corresponds to a constant and female,
education, hours variables in the data.

(d) Find the sample size $n$ of this data and column rank $k$ of the matrix $Xs$

In [5]:
n = len(y)
print(n)
k = X.shape[1]
print(k)

50742
4


(e) Calculate the OLS estimator $\hat \beta$ for $\beta$ using the usual closed-form formula 
$$ \hat \beta = (X^{'}X)^{-1} X^{'}y $$

In [6]:
beta_hat = np.linalg.inv( np.transpose(X) @ X ) @ np.transpose(X) @ y
print(beta_hat)

[ 8.57636643 -0.27818095  0.11322532  0.0143364 ]


(e) Calculate the prediction $\hat y_i = x_i^{'} \beta$ for each given sample point $x_i$. Save the prediction
$\hat y = (\hat y_1, \hat y_2, ..., \hat y_n )^{'}$ in memory for future use in the following steps.

In [7]:
y_hat = X @ beta_hat
y_hat

array([10.58020828, 11.25956018, 10.73497691, ..., 10.65189029,
       10.51707335, 10.39530096])

(g) Calculate the residual $\hat \mu_i = y_i - x_i^{'} \hat \beta$ for each given sample point $x_i$. Save the residual vector $\hat{\mu} = (\hat \mu_1, \hat \mu_2, ..., \hat \mu_n )^{'}$ in memory for future use in the following steps.

In [8]:
mu_hat = y - y_hat
mu_hat

array([ 1.31115362, -0.43978189, -0.36148573, ...,  0.5733531 ,
        0.07956139,  0.60679888])

(h) Calculate the standard error of the regression using only basic arithmetic operations and basic matrix
operations

$$ \sqrt{s^2} = \sqrt{ \frac{\mu^{'} \mu}{n - k} } $$

In [9]:
se = np.sqrt(np.transpose(mu_hat) @ mu_hat /(n - k))
se

0.5946986423377512

(i) Calculate $Est.Var \bigg( \hat \beta \bigg| \{ x_k \}_{k = 1}^n \bigg)$ , the estimated covariance matrix of $\hat \beta$ given data, using only basic arithmetic operations and basic matrix operations.

$$ Est.Var \bigg( \hat \beta \bigg| \{ x_k \}_{k = 1}^n \bigg) = s^2 (X^{'}X)^{-1} $$

In [36]:
EstVar = np.diag([se**2] * k) @ np.linalg.inv( np.transpose(X) @ X )
print(EstVar)

[[ 3.80316414e-04 -2.20563037e-05 -1.08539202e-05 -4.85588278e-06]
 [-2.20563037e-05  2.94610568e-05 -3.63149841e-07  3.32459841e-07]
 [-1.08539202e-05 -3.63149841e-07  9.47326121e-07 -4.98008451e-08]
 [-4.85588278e-06  3.32459841e-07 -4.98008451e-08  1.23389119e-07]]


(j) Calculate the **Total Sum of Squares (TSS), Explained Sum of Squares (ESS), Residual
Sum of Squares (RSS)** using only basic arithmetic operations and basic matrix operations.

$$TSS = \sum_{i = 1}^n (y_i - \bar y)^2 $$
$$RSS = \sum_{i = 1}^n (\hat y_i - \bar y)^2 $$
$$ESS = \sum_{i = 1}^n (y_i - \hat y)^2 $$

In [11]:
y_mean = np.sum(y) / n
TSS = np.sum((y - y_mean)**2)
print(TSS)
RSS = np.sum((y_hat - y_mean)**2)
print(RSS)
ESS = np.sum((y - y_hat)**2)
print(ESS)

24960.672254528137
7016.342635913588
17944.329618614618


(k) Calculate $R^2$. Verify that the following is true:
$$ \frac{ESS}{TSS} = 1 - \frac{RSS}{TSS} $$

$R^2 = \frac{RSS}{TSS} = 0.2811$

In [12]:
RSS / TSS

0.2810959001571261

In [13]:
ESS / TSS

0.7189040998428767

In [14]:
1 - RSS / TSS

0.718904099842874

(l) Numerically calculate $\sum_{i = 1}^n \hat \mu_i $ and $\sum_{i = 1}^n \hat \mu_i x_i $. Compare your results with the theoretical
results.

_Theoretical Results_

$\sum_{i = 1}^n \hat \mu_i = \sum_{i = 1}^n (y_i - \hat y_i) = \sum_{i = 1}^n (y_i - x_i^{'} \hat \beta) = 0 $


$\sum_{i = 1}^n \hat \mu_i x_i = \sum_{i = 1}^n (y_i - \hat y_i) x_i = X^{'} \bigg(y - X(X^{'}X)^{-1}X^{'}y \bigg) = 0 $

_Numerical Results_

In [15]:
np.sum(mu_hat)

-7.379838962151553e-10

In [16]:
out = np.transpose(X) * mu_hat
out.sum(axis = 1)

array([-7.37983896e-10, -6.00280714e-10, -1.10322844e-08, -3.40678525e-08])

(m) Construct the t-statistic corresponding to $H_0: c^{'} \beta = 0$ vs. $H_1: c^{'} \beta \neq 0$ for the
following $c$s, respectively, and calculate the corresponding p-values.

$$ c = 
\begin{pmatrix}
1 \\ 
0 \\
0 \\
0
\end{pmatrix}, 
c = 
\begin{pmatrix}
0 \\ 
1 \\
0 \\
0
\end{pmatrix}, 
c = 
\begin{pmatrix}
0 \\ 
0 \\
1 \\
0
\end{pmatrix}, 
c = 
\begin{pmatrix}
0 \\ 
0 \\
0 \\
1
\end{pmatrix}
$$

Denote $ \beta = 
\begin{pmatrix}
\beta_1 \\ 
\beta_2 \\
\beta_3 \\
\beta_4
\end{pmatrix}
$, we have the limiting distribution of $c^{'} \hat \beta$ under $H_0$ as

$$ 
c^{'} \hat \beta \rightarrow^{d} N \bigg(0, c^{'}  EstVar(\hat \beta) c \bigg)
$$

The corresponding t-statistics are 

$$ t_i = \frac{\hat \beta_i}{ se(\hat \beta_i)}, i \in \{ 1, 2, 3, 4 \} $$
where the standard error of the corresponding estimates are calculated from the diagonal elements of the estimated variance-covariance matrix. 

In [24]:
t_0 = beta_hat[0] / np.sqrt(EstVar[0,0])
t_1 = beta_hat[1] / np.sqrt(EstVar[1,1])
t_2 = beta_hat[2] / np.sqrt(EstVar[2,2])
t_3 = beta_hat[3] / np.sqrt(EstVar[3,3])

The corresponding p-values are shown as follows:

In [31]:
from scipy.stats import norm
p_0 = 2 * (1 - norm.cdf( np.abs(t_0)))
p_1 = 2 * (1 - norm.cdf( np.abs(t_1)))
p_2 = 2 * (1 - norm.cdf( np.abs(t_2)))
p_3 = 2 * (1 - norm.cdf( np.abs(t_3)))

In [35]:
print('t-statistic = %6.3f p-value = %6.4f' % (t_0, p_0))
print('t-statistic = %6.3f p-value = %6.4f' % (t_1, p_1))
print('t-statistic = %6.3f p-value = %6.4f' % (t_2, p_2))
print('t-statistic = %6.3f p-value = %6.4f' % (t_3, p_3))

t-statistic = 439.775 p-value = 0.0000
t-statistic = -51.251 p-value = 0.0000
t-statistic = 116.331 p-value = 0.0000
t-statistic = 40.813 p-value = 0.0000


(n) Construct the t-statistic corresponding to $H_0: c^{'} \beta = 1$ vs. $H_1: c^{'} \beta \neq 1$ for the
following $c$ and calculate the corresponding p-values.

$$c = 
\begin{pmatrix}
1 \\ 
1 \\
1 \\
1
\end{pmatrix}
$$

Given $ c = 
\begin{pmatrix}
1 \\ 
1 \\
1 \\
1
\end{pmatrix}
$, we have the limiting distribution of $c^{'} \hat \beta$ under $H_0$ as

$$ 
c^{'} \hat \beta \rightarrow^{d} N \bigg(0, c^{'}  EstVar(\hat \beta) c \bigg)
$$

The corresponding t-statistic is 

$$ t = \frac{c^{'} \hat \beta}{ \sqrt{c^{'}  EstVar(\hat \beta) c} } $$


In [44]:
c = np.ones(4)
se = np.sqrt(c @ EstVar @ np.transpose(c))
t = c @ beta_hat / se

The corresponding p-value is

In [45]:
p = 2 * (1 - norm.cdf( np.abs(t)))
print('t-statistic = %6.3f p-value = %6.4f' % (t, p))

t-statistic = 460.241 p-value = 0.0000


(o) Now consider minimizing the objective function numerically using IPOPT:

$$ \min_{\beta} \sum_{i = 1}^n (y_i - x_i^{'} \beta)^2$$

Solve the following problem numerically with cyIPOPT by **(i) supplying only objective function and gradient to cyIPOPT**

In [47]:
import cyipopt

class ols:

    def __init__(self):
        pass

    def objective(self, beta):
        #
        # The callback for calculating the objective
        #
        return np.transpose(y - X @ beta) @ (y - X @ beta)

    def gradient(self, beta):
        #
        # The callback for calculating the gradient
        #
        return -2 * np.transpose(X) @ y + 2 * np.transpose(X) @ X @ beta

    def intermediate(
            self,
            alg_mod,
            iter_count,
            obj_value,
            inf_pr,
            inf_du,
            mu,
            d_norm,
            regularization_size,
            alpha_du,
            alpha_pr,
            ls_trials
            ):

        #
        # Example for the use of the intermediate callback.
        #
        print("Objective value at iteration #%d is - %g" % (iter_count, obj_value))


def main():
    #
    # Define the problem
    #
    x0 = [0.0, 0.0, 0.0, 0.0]

    # lb = [1.0, 1.0, 1.0, 1.0]
    # ub = [5.0, 5.0, 5.0, 5.0]

    cl = []
    # cu = []

    nlp = cyipopt.Problem(
        n=len(x0),
        m=len(cl),
        problem_obj=ols(),
        # lb=lb,
        # ub=ub,
        # cl=cl,
        # cu=cu
        )

    #
    # Set solver options
    #
    #nlp.addOption('derivative_test', 'second-order')
    nlp.add_option('mu_strategy', 'adaptive')
    nlp.add_option('tol', 1e-7)
    nlp.add_option('linear_solver', 'ma57')
    nlp.add_option('hessian_approximation', 'limited-memory')
    
    #
    # Scale the problem (Just for demonstration purposes)
    #
    nlp.set_problem_scaling(
        obj_scaling=2,
        x_scaling=[1, 1, 1, 1]
        )
    nlp.add_option('nlp_scaling_method', 'user-scaling')

    #
    # Solve the problem
    #
    x, info = nlp.solve(x0)

    print("Solution of the primal variables: x=%s\n" % repr(x))
    print("Solution of the dual variables: lambda=%s\n" % repr(info['mult_g']))
    print("Objective=%s\n" % repr(info['obj_val']))

if __name__ == '__main__':
    main()


Objective value at iteration #0 is - 5.79418e+06
Objective value at iteration #1 is - 2.47222e+06
Objective value at iteration #2 is - 136348
Objective value at iteration #3 is - 134232
Objective value at iteration #4 is - 86137.6
Objective value at iteration #5 is - 85868.6
Objective value at iteration #6 is - 34961.8
Objective value at iteration #7 is - 30985.8
Objective value at iteration #8 is - 18713.6
Objective value at iteration #9 is - 18631.3
Objective value at iteration #10 is - 17991
Objective value at iteration #11 is - 17957.4
Objective value at iteration #12 is - 17945.6
Objective value at iteration #13 is - 17944.8
Objective value at iteration #14 is - 17944.6
Objective value at iteration #15 is - 17944.6
Objective value at iteration #16 is - 17944.4
Objective value at iteration #17 is - 17944.3
Objective value at iteration #18 is - 17944.3
Objective value at iteration #19 is - 17944.3
Objective value at iteration #20 is - 17944.3
Objective value at iteration #21 is - 17

Solve the following problem numerically with cyIPOPT by **(ii) supplying objective function, gradient, hessian althogether respectively**

In [48]:
import cyipopt

class ols:

    def __init__(self):
        pass

    def objective(self, beta):
        #
        # The callback for calculating the objective
        #
        return np.transpose(y - X @ beta) @ (y - X @ beta)

    def gradient(self, beta):
        #
        # The callback for calculating the gradient
        #
        return - 2 * np.transpose(X) @ y + 2 * np.transpose(X) @ X @ beta

    def hessianstructure(self):
        return np.nonzero(np.tril(np.ones((4, 4))))

    def hessian(self, beta, lagrange, obj_factor):
        #
        # The callback for calculating the Hessian
        #
        
        H = 2 * np.transpose(X) @ X 
        row, col = self.hessianstructure()
        return H[row, col]

    def intermediate(
            self,
            alg_mod,
            iter_count,
            obj_value,
            inf_pr,
            inf_du,
            mu,
            d_norm,
            regularization_size,
            alpha_du,
            alpha_pr,
            ls_trials
            ):

        #
        # Example for the use of the intermediate callback.
        #
        print("Objective value at iteration #%d is - %g" % (iter_count, obj_value))


def main():
    #
    # Define the problem
    #
    x0 = [0.0, 0.0, 0.0, 0.0]

    # lb = [1.0, 1.0, 1.0, 1.0]
    # ub = [5.0, 5.0, 5.0, 5.0]

    cl = []
    # cu = []

    nlp = cyipopt.Problem(
        n=len(x0),
        m=len(cl),
        problem_obj=ols(),
        # lb=lb,
        # ub=ub,
        # cl=cl,
        # cu=cu
        )

    #
    # Set solver options
    #
    #nlp.addOption('derivative_test', 'second-order')
    nlp.add_option('mu_strategy', 'adaptive')
    nlp.add_option('tol', 1e-7)
    nlp.add_option('linear_solver', 'ma57')
    nlp.add_option('hessian_approximation', 'exact')
    #
    # Scale the problem (Just for demonstration purposes)
    #
    nlp.set_problem_scaling(
        obj_scaling=2,
        x_scaling=[1, 1, 1, 1]
        )
    nlp.add_option('nlp_scaling_method', 'user-scaling')

    #
    # Solve the problem
    #
    x, info = nlp.solve(x0)

    print("Solution of the primal variables: x=%s\n" % repr(x))

    print("Solution of the dual variables: lambda=%s\n" % repr(info['mult_g']))

    print("Objective=%s\n" % repr(info['obj_val']))


if __name__ == '__main__':
    main()


Objective value at iteration #0 is - 5.79418e+06
Objective value at iteration #1 is - 17944.3
Solution of the primal variables: x=array([ 8.57636643, -0.27818095,  0.11322532,  0.0143364 ])

Solution of the dual variables: lambda=array([], dtype=float64)

Objective=17944.32961861461



**The estimation results coincide. However, the provision of Hession matrix enables the estimation procedure to be completed using 1 iteration whereas no-Hession procedure takes 32 iterations.**

## _2 (Python) (Weak) Law of Large Numbers_

(a) Consider a continuous random variable $X_i \sim Uniform[0,2]$. What is $E[X_i]$ and $Var[X_i]$?

(b) Consider the $X_i$ defined above in (a) for $i = 1, 2, ..., n$, where each $X_i$ is uncorrelated with $X_j$ whenever $i \neq j$. Consider the sample mean

$$\bar X_n = \frac{1}{n} \sum_{i = 1}^n X_i$$

What is $E[X_i]$ and $Var[X_i]$?

Now repeat the following (c) -- (e) for $n = 1, 2, 3, 5, 10, 50, 100, 1000, 3000$. Parallelize your routine
for (c)-(e) using **joblib**.

(c) (Python) Generate a size n vector of independent $Uniform[0,2]$ random variables and
calculate its sample mean $\bar X_n$.

(d) (Python) Take $| \bar X_n - E[X_i] |$ and report the value.

(e) (Python) Now consider a continuous transformation


$$f(x) = 2 x^2 - 5x + 1 + \frac{1}{3x}$$

Take $| \bar X_n - E[X_i] |$ and report the value.

(f) What happens to the reported value in (d) and (e) as n increases? Discuss.

## _3 (Python) Central Limit Theorem_

(a) Consider a continuous random variable $X_i \sim Uniform[0,2]$. What is $E[X_i]$ and $Var[X_i]$?

(b) Consider the $X_i$ defined above in (a) for $i = 1, 2, ..., n$, where each $X_i$ is uncorrelated with $X_j$ whenever $i \neq j$. Consider the sample mean

$$\bar X_n = \frac{1}{n} \sum_{i = 1}^n X_i$$

What is $E[X_i]$ and $Var[X_i]$?

(c) Consider the transformation

$$Y_n = \sqrt{n} (\bar X_n - E[X_i]) $$

What is $E[X_i]$ and $Var[X_i]$?

(c) Consider the transformation

$$Y_n = \sqrt{n} \frac{\bar X_n - E[X_i]}{\sqrt{Var[X_i]}} $$

What is $E[X_i]$ and $Var[X_i]$?

Now repeat the following for $n = 1, 2, 3, 5, 10, 50, 100, 1000, 3000$. Parallelize your routine
for (e)-(m) using **joblib**.

(e) (Python) Generate $t = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..., 2500$ $n × 1$ vectors of independent
$Uniform[0,2]$ random variables and calculate its sample mean $\bar X_n^t$ respectively for each $t$.
Denote this size 2500 vector as


$$v_n^{2500} = \bigg( \bar X_n^1, \bar X_n^2, ..., \bar X_n^{2500} \bigg)$$

for now.

(f) (Python) Calculate the mean and variance of $v^{2500}$ and report.

**g()**

(h) (Python) Calculate the mean and variance of $y^{2500}$ and report.

(i) (Python) Plot the histogram of $y^{2500}$ and report.

**j()**

(k) (Python) Calculate the mean and variance of $z^{2500}$ and report.

(l) (Python) Plot the histogram of $z^{2500}$ and report.

(m) What happens to the reported values in (f), (h), (k) and histograms in (i) and (l) as n
increases? Discuss.

## _4 (Python) WLLN with Simple Regression_##

In this exercise, you will generate datasets for simple regression yourself, and then try to esti-
mate the model parmaters to examine the properties of simple regression OLS estimators as the
sample size $n$ grows.
Repeat the following for $n = 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 50, 75, 100, 250, 500, 1000, 2000, 3000$.
Parallelize your routine for (a)-(e) using **joblib**.

(a) (Python) Generate a $n × 1$ column vector of $Uniform[0, 12]$ random variable and denote it as $x$.

(b) (Python) Generate a $n × 1$ column vector of $Uniform[− 4,4]$ random variable and denote it as $u$.

(c) (Python) Generate the y vector using the following formula:

$$y_i = 3 + 2 x_i + u_i$$

for each $i = 1, 2, 3, ..., n$. That is, $i$’th row of $x$ and $u$ corresponds to $i$’th observation.

(d) (Python) Now you have a Monte-Carlo dataset of size $n$. Estimate the $\beta$ in the following model

$$y_i = \alpha + \beta x_i + u_i$$

using OLS. (Recall the formula $\frac{\hat Cov(x_i, y_i)}{\hat Var(x_i)}$) What is the calculated value of $\hat \beta_{OLS, n}$? Report.

(e) What happens to $|\hat \beta_{OLS, n} - 2|$  as $n$ increases? Discuss.

## _5 (Python) CLT with Simple Regression_ ##

In this exercise, you will generate datasets for simple regression yourself, and then try to esti-
mate the model parmaters to examine the properties of simple regression OLS estimators as the
sample size $n$ grows.
Repeat the following for $n = 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 50, 75, 100, 250, 500, 1000, 2000, 3000$.
Parallelize your routine for (a)-(e) using **joblib**.

(a) (Python) Generate a $n × 1$ column vector of $Uniform[0, 12]$ random variable and denote it as $x$.

(b) (Python) Generate a $n × 1$ column vector of $Uniform[− 4,4]$ random variable and denote it as $u$.

(c) (Python) Generate the y vector using the following formula:

$$y_i = 3 + 2 x_i + u_i$$

for each $i = 1, 2, 3, ..., n$. That is, $i$’th row of $x$ and $u$ corresponds to $i$’th observation.

(d) (Python) Now you have a Monte-Carlo dataset of size $n$. Estimate the $\beta$ in the following model

$$y_i = \alpha + \beta x_i + u_i$$

using OLS. (Recall the formula $\frac{\hat Cov(x_i, y_i)}{\hat Var(x_i)}$). Save it in the memory.